In [351]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer
from functools import reduce
from scipy.spatial.distance import pdist, squareform, cdist
import jgraph
from cyjupyter import Cytoscape
from scipy.stats import kendalltau
from scipy.stats import weightedtau
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sn

In [352]:
connection = mysql.connector.connect(host='127.0.0.1',
                                         database='employment',
                                         user='root',
                                         password='thanhn123')

In [353]:
def get_data(connection, descriptor_domain,LV_only = False, filt = False):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    descriptor_domeain:
        work context
        knowledge
        skills
        gwas
        job_zones, filter
        Abilities, filter
    
    '''
    df= pd.DataFrame(data=None)
    if descriptor_domain == 'work_context':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_context
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'CX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'job_zones':
        if filt:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
            df = df[df['job_zone'].isin(val)]
        else:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
    elif descriptor_domain == 'abilities':
        if filt:
#             df= pd.read_sql('''
#             SELECT title, element_name, element_id,data_value FROM employment.''' + descriptor_domain + '''
#             LEFT JOIN employment.content_model_reference USING (element_id) 
#             LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
#             df = df[df['element_id'].contains(val)]
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE element_id LIKE\''''+ val +'''%\'; ''', con = connection)
            
        else:
            df= pd.read_sql('''
            SELECT title, element_name,element_id, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_values':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_values
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'EX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'interests':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.interests
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'OI';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_styles':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_styles
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'IM';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'name':
        df= pd.read_sql('''
            SELECT * FROM employment.occupation_data;''', con = connection)
    else:
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        if LV_only:
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE \"LV\";''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    return df

In [354]:
def distance_table(df, jz=False):
    
#     squareform(pdist(df.iloc[:, 1:])), columns=df.title.unique(), index=df.title.unique()
    dist = []
    name_dict = {}
    if jz:
        name_dict = df['title'].sort_values().reset_index()['title'].to_dict()
        dist = pdist(df[df.columns[1:]],'euclidean')
    else:
        for i in range(len(df.index)):
            name_dict[i] = df.index[i]
        dist = pdist(df,'euclidean') #Euclidean distance for 1XN
    df_dist = pd.DataFrame(squareform(dist)) #Euclidean distance for NxN
    return df_dist
    columns = list(df_dist.index) 
    for i in range(len(df.index)): #convert 0 values to NaN
        df_dist.loc[i,i] = np.NaN
    mean = df_dist.mean().mean()
    std = df_dist.stack().std() 

    for col in columns: #Z-score calculated 
        df_dist[col] = (df_dist[col] - mean)/std
#     return df_dist
    return df_dist.rename(columns = name_dict, index = name_dict)

In [355]:
df_cps = pd.read_csv('../CPS Job Changes/JobChanges_2011to19.csv')

In [356]:
info_dict = {'descriptor_domain':['abilites','work_context','interests',
                               'knowledge','skills','work_styles','work_values','gwas','job_zones'],
            'starter':['IM,LV',None,'OI',None,None,'IM','EX',None,None],
            'change':[None,'CX',None,'IM, LV','IM, LV',None,None,'IM, LV','JZ']}
df_info = pd.DataFrame(data = info_dict)
df_info

,descriptor_domain,starter,change
0,abilites,"IM,LV",None
1,work_context,None,CX
2,interests,OI,None
3,knowledge,None,"IM, LV"
4,skills,None,"IM, LV"
5,work_styles,IM,None
6,work_values,EX,None
7,gwas,None,"IM, LV"
8,job_zones,None,JZ


In [357]:
df_ab = get_data(connection,'abilities')
df_in = get_data(connection,'interests')
df_ws = get_data(connection,'work_styles')
df_wv = get_data(connection, 'work_values')
df_wc = get_data(connection,'work_context')
df_knowledge = get_data(connection,'knowledge')
df_gwas = get_data(connection,'work_activities')
df_jz = get_data(connection,'job_zones')
df_jz = df_jz[df_jz['title'] != 'Legislators']
df_skills = get_data(connection,'skills')

In [358]:
# wc_dist = distance_table(df_wc)
# knowledge_dist = distance_table(df_knowledge)
# skills_dist = distance_table(df_skills)
# gwas_dist = distance_table(df_gwas)
# jz_dist = distance_table(df_jz,True)
# wv_dist = distance_table(df_wv)
# in_dist = distance_table(df_in)
# ab_dist = distance_table(df_ab)
# ws_dist = distance_table(df_ws)
# df_concat = pd.concat([wc_dist,knowledge_dist,skills_dist,gwas_dist,jz_dist.multiply(1.3)]).groupby(level =0).mean()

In [359]:
distance_table(df_skills)

,0,1,2,3,4,5,6,7,8,9,...,958,959,960,961,962,963,964,965,966,967
0,0.000000,5.249560,3.373426,3.478606,4.611182,3.794664,7.816019,3.679062,3.543173,3.688990,...,7.050195,8.630268,2.819849,4.149949,5.091540,3.535336,7.768472,8.382183,5.478474,3.929790
1,5.249560,0.000000,7.820874,4.057398,6.932415,5.011297,7.937476,4.659182,5.779243,3.907144,...,6.307351,8.020873,6.060947,7.586617,7.698588,6.712941,8.638193,8.170320,4.403328,6.623562
2,3.373426,7.820874,0.000000,5.519710,5.144657,5.152308,9.379757,4.932221,4.806914,5.755702,...,9.081225,10.364506,3.854167,3.932477,5.520102,4.118604,8.807332,9.975089,7.973674,4.357947
3,3.478606,4.057398,5.519710,0.000000,3.799428,3.025025,6.524925,3.877615,4.180715,3.415626,...,5.604886,7.292638,4.321730,4.662052,5.358482,4.380400,6.753647,7.240815,4.692124,3.643652
4,4.611182,6.932415,5.144657,3.799428,0.000000,3.433799,7.247924,5.202917,4.109887,5.156433,...,6.957489,8.351447,4.636728,3.232023,3.951737,3.712351,6.704099,8.238052,7.403261,2.770487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,3.535336,6.712941,4.118604,4.380400,3.712351,3.903950,7.393430,4.747605,2.235839,5.155221,...,7.192100,8.425938,2.236314,3.968662,2.685354,0.000000,6.896992,8.297111,6.574076,3.534848
964,7.768472,8.638193,8.807332,6.753647,6.704099,7.528136,3.278986,8.923797,7.246556,8.212728,...,4.202636,4.079813,7.953609,6.745797,5.311749,6.896992,0.000000,3.871518,7.421312,7.086332
965,8.382183,8.170320,9.975089,7.240815,8.238052,8.681096,2.665483,9.616013,8.289350,8.791786,...,2.465269,2.420139,8.807542,7.981449,7.396376,8.297111,3.871518,0.000000,5.982531,8.470390
966,5.478474,4.403328,7.973674,4.692124,7.403261,6.474512,5.967313,6.609964,6.120854,5.666606,...,4.730827,6.276974,6.334353,7.121826,7.425305,6.574076,7.421312,5.982531,0.000000,6.873192


In [360]:
def exact_difference(df):
    '''
    return adjacency matrix difference. Difference is based on column - row
    zeroed out 
    '''
    ini_array1 = df.values - df.values[:, None]
    arr = ini_array1[ini_array1 <0] = 0
    arr = np.sqrt(np.mean(np.power(ini_array1,1),axis =2))
#     return arr
    new_df = pd.concat((df.reset_index()['title'], pd.DataFrame(arr, columns=df.reset_index()['title'])), axis=1)
    return new_df

In [361]:
# ini_array1 = df_skills.values - df_skills.values[:, None]
# arr = np.where(ini_array1<0, 0, ini_array1) 
# arr = np.mean(np.power(arr,2),axis =2) #Euclidean distance for 1XN
# pd.concat((df_skills.reset_index()['title'], pd.DataFrame(arr, columns=df_skills.reset_index()['title'])), axis=1)

## CELL -> Transition from Column - Row. Another way of saying this is 
## 'what avg attributes measure do we need to transition from the occupation in row to column'

### Example:



In [362]:
exact_difference(df_skills)

,title,Accountants,Actors,Actuaries,Acupuncturists,Acute Care Nurses,Adapted Physical Education Specialists,Adhesive Bonding Machine Operators and Tenders,"Administrative Law Judges, Adjudicators, and Hearing Officers",Administrative Services Managers,...,"Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders",Wellhead Pumpers,"Wholesale and Retail Buyers, Except Farm Products",Wind Energy Engineers,Wind Energy Operations Managers,Wind Energy Project Managers,Wind Turbine Service Technicians,"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",Word Processors and Typists,Zoologists and Wildlife Biologists
0,Accountants,0.000000,0.163881,0.620714,0.382473,0.711638,0.570213,0.697854,0.513809,0.545763,...,0.603561,0.698161,0.485357,0.658461,0.775979,0.632907,0.782396,0.724667,0.307292,0.603561
1,Actors,0.743063,0.000000,0.960580,0.688061,0.984378,0.834780,0.827820,0.766346,0.860565,...,0.708015,0.796151,0.827043,0.972772,1.030257,0.928132,0.970861,0.835549,0.522494,0.916203
2,Actuaries,0.112758,0.157661,0.000000,0.348466,0.597495,0.457165,0.681175,0.353755,0.457009,...,0.588582,0.685253,0.351121,0.504834,0.674960,0.504268,0.745079,0.698365,0.252982,0.453242
3,Acupuncturists,0.487560,0.198926,0.765133,0.000000,0.731535,0.570589,0.651591,0.581132,0.613654,...,0.529690,0.631099,0.596178,0.739208,0.822974,0.693645,0.763918,0.685565,0.242899,0.666976
4,Acute Care Nurses,0.250428,0.000000,0.534656,0.000000,0.000000,0.219740,0.525765,0.370906,0.341426,...,0.398569,0.531037,0.373784,0.423084,0.533586,0.419183,0.600357,0.549545,0.146385,0.305193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,Wind Energy Project Managers,0.270977,0.094868,0.547462,0.250143,0.540635,0.373210,0.588096,0.404969,0.280051,...,0.477793,0.592814,0.207364,0.517135,0.505117,0.000000,0.669008,0.607924,0.102817,0.416276
964,Wind Turbine Service Technicians,0.578298,0.373593,0.806226,0.463064,0.725554,0.681700,0.208395,0.690548,0.660087,...,0.043095,0.194202,0.650933,0.693336,0.726439,0.704982,0.000000,0.175662,0.223926,0.679075
965,"Woodworking Machine Setters, Operators, and Te...",0.855737,0.616441,1.033164,0.765320,0.976144,0.916281,0.456853,0.915111,0.916983,...,0.347234,0.391335,0.912140,0.955211,0.965771,0.950714,0.718132,0.000000,0.415245,0.941048
966,Word Processors and Typists,0.828251,0.582482,1.013974,0.747759,1.027966,0.909631,0.771177,0.886405,0.909160,...,0.667083,0.728011,0.885357,0.993766,1.061939,0.963995,0.958868,0.746229,0.000000,0.953415


In [363]:
## standardization: TODO

# def distance_table_from_euclidean(df, jz=False):
#     '''
#     Return standardized ONET measures based on Euclidean measures
#     '''

#     dist = []
#     name_dict = {}
#     df_dist = df #Euclidean distance for NxN
# #     return df_dist
#     columns = list(df_dist.index) 
#     for i in range(len(df.index)):
#         name_dict[i] = df.index[i]
#     for i in range(len(df.index)): #convert 0 values to NaN
#         df_dist.loc[i,i] = np.NaN
#     mean = df_dist.mean().mean()
#     std = df_dist.stack().std() 

#     for col in columns: #Z-score calculated 
#         df_dist[col] = (df_dist[col] - mean)/std
# #     return df_dist
#     return df_dist.rename(columns = name_dict, index = name_dict)

In [364]:
# df_skills #zero differences if < 0

In [365]:
# distance_table(df_skills)

In [366]:
difference_abilites = exact_difference(df_ab)
# difference_in = exact_difference(df_in)
# difference_ws = exact_difference(df_ws)
# difference_wv =exact_difference(df_wv)
# difference_wc =exact_difference(df_wc)
# difference_knowledge =exact_difference(df_knowledge)
difference_gwas =exact_difference(df_gwas)
# difference_jz =exact_difference(df_jz)
difference_skills =exact_difference(df_skills)

In [367]:
difference_skills

,title,Accountants,Actors,Actuaries,Acupuncturists,Acute Care Nurses,Adapted Physical Education Specialists,Adhesive Bonding Machine Operators and Tenders,"Administrative Law Judges, Adjudicators, and Hearing Officers",Administrative Services Managers,...,"Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders",Wellhead Pumpers,"Wholesale and Retail Buyers, Except Farm Products",Wind Energy Engineers,Wind Energy Operations Managers,Wind Energy Project Managers,Wind Turbine Service Technicians,"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",Word Processors and Typists,Zoologists and Wildlife Biologists
0,Accountants,0.000000,0.163881,0.620714,0.382473,0.711638,0.570213,0.697854,0.513809,0.545763,...,0.603561,0.698161,0.485357,0.658461,0.775979,0.632907,0.782396,0.724667,0.307292,0.603561
1,Actors,0.743063,0.000000,0.960580,0.688061,0.984378,0.834780,0.827820,0.766346,0.860565,...,0.708015,0.796151,0.827043,0.972772,1.030257,0.928132,0.970861,0.835549,0.522494,0.916203
2,Actuaries,0.112758,0.157661,0.000000,0.348466,0.597495,0.457165,0.681175,0.353755,0.457009,...,0.588582,0.685253,0.351121,0.504834,0.674960,0.504268,0.745079,0.698365,0.252982,0.453242
3,Acupuncturists,0.487560,0.198926,0.765133,0.000000,0.731535,0.570589,0.651591,0.581132,0.613654,...,0.529690,0.631099,0.596178,0.739208,0.822974,0.693645,0.763918,0.685565,0.242899,0.666976
4,Acute Care Nurses,0.250428,0.000000,0.534656,0.000000,0.000000,0.219740,0.525765,0.370906,0.341426,...,0.398569,0.531037,0.373784,0.423084,0.533586,0.419183,0.600357,0.549545,0.146385,0.305193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,Wind Energy Project Managers,0.270977,0.094868,0.547462,0.250143,0.540635,0.373210,0.588096,0.404969,0.280051,...,0.477793,0.592814,0.207364,0.517135,0.505117,0.000000,0.669008,0.607924,0.102817,0.416276
964,Wind Turbine Service Technicians,0.578298,0.373593,0.806226,0.463064,0.725554,0.681700,0.208395,0.690548,0.660087,...,0.043095,0.194202,0.650933,0.693336,0.726439,0.704982,0.000000,0.175662,0.223926,0.679075
965,"Woodworking Machine Setters, Operators, and Te...",0.855737,0.616441,1.033164,0.765320,0.976144,0.916281,0.456853,0.915111,0.916983,...,0.347234,0.391335,0.912140,0.955211,0.965771,0.950714,0.718132,0.000000,0.415245,0.941048
966,Word Processors and Typists,0.828251,0.582482,1.013974,0.747759,1.027966,0.909631,0.771177,0.886405,0.909160,...,0.667083,0.728011,0.885357,0.993766,1.061939,0.963995,0.958868,0.746229,0.000000,0.953415


In [368]:
difference_skills.index = difference_skills.title 
difference_abilites.index = difference_abilites.title 
difference_gwas.index = difference_gwas.title 

In [369]:
difference_skills = difference_skills.drop(['title'],axis =1)
difference_abilites = difference_abilites.drop(['title'],axis =1)
difference_gwas = difference_gwas.drop(['title'],axis =1)

In [370]:
df_concat_exact_difference = pd.concat([difference_gwas,difference_skills,difference_abilites]).groupby(level =0).mean()

In [371]:
# distance_table_from_euclidean(df_concat_exact_difference)

In [372]:
dict_test = {}
for val in df_concat_exact_difference:
    print(val)
    break

Accountants


In [373]:
df_concat_exact_difference

,Accountants,Actors,Actuaries,Acupuncturists,Acute Care Nurses,Adapted Physical Education Specialists,Adhesive Bonding Machine Operators and Tenders,"Administrative Law Judges, Adjudicators, and Hearing Officers",Administrative Services Managers,Adult Basic and Secondary Education and Literacy Teachers and Instructors,...,"Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders",Wellhead Pumpers,"Wholesale and Retail Buyers, Except Farm Products",Wind Energy Engineers,Wind Energy Operations Managers,Wind Energy Project Managers,Wind Turbine Service Technicians,"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",Word Processors and Typists,Zoologists and Wildlife Biologists
title,,,,,,,,,,,,,,,,,,,,,
Accountants,0.000000,0.420091,0.559042,0.634917,0.754756,0.696087,0.801600,0.474580,0.654853,0.376631,...,0.738086,0.795497,0.473168,0.596133,0.819277,0.632872,0.879555,0.765085,0.358921,0.766637
Actors,0.774630,0.000000,0.918360,0.833314,0.971689,0.876358,0.878180,0.751688,0.892346,0.656749,...,0.809501,0.905985,0.769041,0.922807,1.020750,0.914357,1.024506,0.803493,0.583827,0.992453
Actuaries,0.273793,0.444126,0.000000,0.632520,0.713352,0.663422,0.825076,0.393304,0.631104,0.337544,...,0.765577,0.819888,0.468263,0.537459,0.783145,0.572138,0.891196,0.790147,0.415029,0.692225
Acupuncturists,0.424109,0.279570,0.633887,0.000000,0.623874,0.522376,0.625803,0.465736,0.534213,0.336470,...,0.542401,0.613178,0.419109,0.569607,0.704802,0.583242,0.737890,0.569105,0.249855,0.661513
Acute Care Nurses,0.270927,0.203575,0.490484,0.318883,0.000000,0.361529,0.596554,0.325466,0.386375,0.188113,...,0.521752,0.587010,0.312754,0.416652,0.562041,0.418760,0.678227,0.545859,0.192677,0.495477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind Energy Project Managers,0.294690,0.321137,0.488857,0.500179,0.596158,0.545063,0.699390,0.342091,0.439054,0.257292,...,0.632342,0.692389,0.289095,0.447020,0.604474,0.000000,0.779697,0.655228,0.241809,0.572791
Wind Turbine Service Technicians,0.523530,0.361908,0.716892,0.515148,0.667783,0.558905,0.231776,0.596843,0.586120,0.453418,...,0.112878,0.251077,0.492796,0.577936,0.638008,0.648357,0.000000,0.162586,0.276686,0.669793
"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",0.827724,0.583239,0.967387,0.811102,0.939194,0.856545,0.510262,0.838659,0.879404,0.756636,...,0.448412,0.585725,0.805440,0.876262,0.938669,0.916037,0.786422,0.000000,0.504718,0.949129


In [374]:
df_concat_exact_difference.loc[['Accountants','Registered Nurses','Retail Salespersons','Janitors and Cleaners, Except Maids and Housekeeping Cleaners']]

,Accountants,Actors,Actuaries,Acupuncturists,Acute Care Nurses,Adapted Physical Education Specialists,Adhesive Bonding Machine Operators and Tenders,"Administrative Law Judges, Adjudicators, and Hearing Officers",Administrative Services Managers,Adult Basic and Secondary Education and Literacy Teachers and Instructors,...,"Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders",Wellhead Pumpers,"Wholesale and Retail Buyers, Except Farm Products",Wind Energy Engineers,Wind Energy Operations Managers,Wind Energy Project Managers,Wind Turbine Service Technicians,"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",Word Processors and Typists,Zoologists and Wildlife Biologists
title,,,,,,,,,,,,,,,,,,,,,
Accountants,0.000000,0.420091,0.559042,0.634917,0.754756,0.696087,0.801600,0.474580,0.654853,0.376631,...,0.738086,0.795497,0.473168,0.596133,0.819277,0.632872,0.879555,0.765085,0.358921,0.766637
Registered Nurses,0.303549,0.186621,0.517991,0.256050,0.374616,0.341216,0.538784,0.343781,0.424853,0.201518,...,0.468617,0.543754,0.300545,0.448546,0.561458,0.423587,0.627088,0.503216,0.165865,0.499524
Retail Salespersons,0.649461,0.375553,0.824049,0.687673,0.838271,0.753079,0.733234,0.649598,0.741339,0.537787,...,0.669266,0.771380,0.602623,0.777242,0.897172,0.774988,0.896087,0.653196,0.382196,0.865751
"Janitors and Cleaners, Except Maids and Housekeeping Cleaners",0.933592,0.635254,1.067043,0.940057,1.072911,0.973353,0.834699,0.929251,1.002918,0.850165,...,0.759747,0.862748,0.912141,1.002679,1.096960,1.032606,1.033899,0.720117,0.636722,1.087086


In [375]:
# np.mean(df_concat_exact_difference.loc['Accountants'] - df_concat_exact_difference.loc['Actors'])

In [376]:
df_cps

,ONET18_SOC_LY,ONET18_SOC,ONET18_Title,ONET18_Title_LY,count,pct_tot,count_socly,pct_socly,in_grp,in_minor_grp,in_broad_occ,employment_projection_NewJob,wage_change_BetweenJobs
0,11-1011,11-1021,General and Operations Managers,Chief Executives,327891.330000,0.196284,644769.200000,50.854062,1,1,0,5.8,-45.365359
1,11-1011,11-1031,Legislators,Chief Executives,4069.996250,0.002436,644769.200000,0.631233,1,1,0,6.2,-84.131579
2,11-1011,11-2021,Marketing Managers,Chief Executives,5621.145000,0.003365,644769.200000,0.871807,1,0,0,6.7,-25.811908
3,11-1011,11-2022,Sales Managers,Chief Executives,5621.145000,0.003365,644769.200000,0.871807,1,0,0,3.5,-31.337393
4,11-1011,11-3012,Administrative Services Managers,Chief Executives,1832.805000,0.001097,644769.200000,0.284258,1,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115312,53-7199,53-7065,Stockers and Order Fillers,"Material Moving Workers, All Other",4878.226667,0.002920,43481.721667,11.219028,1,0,0,0.7,-13.874346
115313,53-7199,53-7071,Gas Compressor and Gas Pumping Station Operators,"Material Moving Workers, All Other",35.795833,0.000021,43481.721667,0.082324,1,0,0,-2.1,108.180628
115314,53-7199,53-7072,"Pump Operators, Except Wellhead Pumpers","Material Moving Workers, All Other",35.795833,0.000021,43481.721667,0.082324,1,0,0,9.7,41.688482
115315,53-7199,53-7073,Wellhead Pumpers,"Material Moving Workers, All Other",35.795833,0.000021,43481.721667,0.082324,1,0,0,4.2,73.167539


In [377]:
#Distance, ground truth, removed, goodness

In [378]:
good_dict = {}
i = 0
for row in df_cps.iterrows():

    #(ONET18_Title_LY,ONET18_Title) = [employment_projection_NewJob,wage_change_BetweenJobs]
    good_dict[(row[1][3],row[1][2])] = [row[1][-2],row[1][-1]]

In [388]:
df_cps[df_cps['ONET18_Title_LY'] == 'Retail Salespersons'].sort_values(by = 'pct_tot', ascending = False)

,ONET18_SOC_LY,ONET18_SOC,ONET18_Title,ONET18_Title_LY,count,pct_tot,count_socly,pct_socly,in_grp,in_minor_grp,in_broad_occ,employment_projection_NewJob,wage_change_BetweenJobs
77839,41-2031,41-2011,Cashiers,Retail Salespersons,336088.715000,0.201191,4726343.07,7.110967,1,1,0,-7.4,-6.342669
77840,41-2031,41-2012,Gambling Change Persons and Booth Cashiers,Retail Salespersons,336088.715000,0.201191,4726343.07,7.110967,1,1,0,2.5,1.729819
77844,41-2031,41-3021,Insurance Sales Agents,Retail Salespersons,215727.550000,0.129140,4726343.07,4.564365,1,0,0,5.5,101.729819
77847,41-2031,41-3091,"Sales Representatives of Services, Except Adve...",Retail Salespersons,169265.980000,0.101327,4726343.07,3.581331,1,0,0,6.0,122.322900
77857,41-2031,41-9099,"Sales and Related Workers, All Other",Retail Salespersons,159803.040000,0.095662,4726343.07,3.381114,1,0,0,1.9,26.029654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77702,41-2031,27-4015,Lighting Technicians,Retail Salespersons,34.758333,0.000021,4726343.07,0.000735,0,0,0,NaN,NaN
77701,41-2031,27-4014,Sound Engineering Technicians,Retail Salespersons,34.758333,0.000021,4726343.07,0.000735,0,0,0,6.0,116.803954
77700,41-2031,27-4012,Broadcast Technicians,Retail Salespersons,34.758333,0.000021,4726343.07,0.000735,0,0,0,3.1,60.708402
77699,41-2031,27-4011,Audio and Video Technicians,Retail Salespersons,34.758333,0.000021,4726343.07,0.000735,0,0,0,12.3,81.795717


In [382]:
def get_df_comparison(df_onet,df_cps,good_dict,occ,top):
    '''
    return df of comparisons of different occupations
    
    '''
    comparison = {'ONET_occ_before':[],'ONET_occ_after':[],'ONET_employment_proj' : [],
                       'ONET_wage_change':[],'CPS_occ_before': [],'CPS_occ_after': [],'CPS_employment_proj' : [],'CPS_wage_change':[]}
    
    df_cps_iter = df_cps[df_cps['ONET18_Title_LY'] == occ].sort_values(by = 'pct_tot', ascending = False)[0:top]
    df_onet_iter = df_onet.loc[occ].sort_values()[1:top+1].to_frame()
    
    for row in df_cps_iter.iterrows():
        if (occ,row[1][2]) not in good_dict:
            comparison['CPS_occ_before'].append(occ)
            comparison['CPS_occ_after'].append(row[1][2])
            comparison['CPS_employment_proj'].append(None)
            comparison['CPS_wage_change'].append(None)
        else:
            comparison['CPS_occ_before'].append(occ)
            comparison['CPS_occ_after'].append(row[1][2])
            comparison['CPS_employment_proj'].append(good_dict[(occ,row[1][2])][0])
            comparison['CPS_wage_change'].append(good_dict[(occ,row[1][2])][1])
    for row in df_onet_iter.iterrows():
        if (occ,row[0]) not in good_dict:
            comparison['ONET_occ_before'].append(occ)
            comparison['ONET_occ_after'].append(row[0])
            comparison['ONET_employment_proj'].append(None)
            comparison['ONET_wage_change'].append(None)
        else:
            comparison['ONET_occ_before'].append(occ)
            comparison['ONET_occ_after'].append(row[0])
            comparison['ONET_employment_proj'].append(good_dict[(occ,row[0])][0])
            comparison['ONET_wage_change'].append(good_dict[(occ,row[0])][1])
    return pd.DataFrame(comparison)


In [383]:
df_onet =  df_concat_exact_difference

In [384]:
get_df_comparison(df_onet,df_cps,good_dict,'Registered Nurses',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Registered Nurses,"Locker Room, Coatroom, and Dressing Room Atten...",NaN,NaN,Registered Nurses,Physical Therapists,18.2,22.020431
1,Registered Nurses,Teacher Assistants,NaN,NaN,Registered Nurses,Respiratory Therapists,19.4,-16.345062
2,Registered Nurses,"Office Clerks, General",NaN,NaN,Registered Nurses,"Therapists, All Other",14.6,-28.178207
3,Registered Nurses,Childcare Workers,1.7,-66.940976,Registered Nurses,Exercise Physiologists,11.3,-32.917140
4,Registered Nurses,Insurance Policy Processing Clerks,NaN,NaN,Registered Nurses,Speech-Language Pathologists,24.9,7.945516
5,Registered Nurses,Proofreaders and Copy Markers,NaN,NaN,Registered Nurses,Occupational Therapists,15.9,15.891033
6,Registered Nurses,License Clerks,NaN,NaN,Registered Nurses,Nurse Midwives,11.6,43.303065
7,Registered Nurses,Telephone Operators,NaN,NaN,Registered Nurses,Nurse Practitioners,52.4,49.829739
8,Registered Nurses,"Counter Attendants, Cafeteria, Food Concession...",NaN,NaN,Registered Nurses,Licensed Practical and Licensed Vocational Nurses,9.1,-35.215664
9,Registered Nurses,Insurance Claims Clerks,NaN,NaN,Registered Nurses,Nurse Anesthetists,13.7,138.450624


In [385]:
get_df_comparison(df_onet,df_cps,good_dict,'Retail Salespersons',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Retail Salespersons,"Locker Room, Coatroom, and Dressing Room Atten...",8.1,-0.576606,Retail Salespersons,Cashiers,-7.4,-6.342669
1,Retail Salespersons,"Hosts and Hostesses, Restaurant, Lounge, and C...",8.2,-8.566722,Retail Salespersons,Gambling Change Persons and Booth Cashiers,2.5,1.729819
2,Retail Salespersons,Telemarketers,-14.2,4.118616,Retail Salespersons,Insurance Sales Agents,5.5,101.729819
3,Retail Salespersons,Models,0.5,12.273476,Retail Salespersons,"Sales Representatives of Services, Except Adve...",6.0,122.322900
4,Retail Salespersons,Gaming Dealers,NaN,NaN,Retail Salespersons,"Sales and Related Workers, All Other",1.9,26.029654
5,Retail Salespersons,"Ushers, Lobby Attendants, and Ticket Takers",3.3,-6.919275,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",4.1,220.840198
6,Retail Salespersons,Manicurists and Pedicurists,19.2,2.059308,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",0.9,137.314662
7,Retail Salespersons,"Cooks, Fast Food",-13.4,-6.919275,Retail Salespersons,Real Estate Sales Agents,2.3,93.822076
8,Retail Salespersons,Teacher Assistants,NaN,NaN,Retail Salespersons,Real Estate Brokers,2.6,136.490939
9,Retail Salespersons,"Counter Attendants, Cafeteria, Food Concession...",NaN,NaN,Retail Salespersons,Receptionists and Information Clerks,3.6,19.028007


In [386]:
get_df_comparison(df_onet,df_cps,good_dict,'Janitors and Cleaners, Except Maids and Housekeeping Cleaners',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,"Janitors and Cleaners, Except Maids and Housek...",Models,0.5,3.335861,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061
1,"Janitors and Cleaners, Except Maids and Housek...","Locker Room, Coatroom, and Dressing Room Atten...",8.1,-8.491281,"Janitors and Cleaners, Except Maids and Housek...",Landscaping and Groundskeeping Workers,10.1,10.917362
2,"Janitors and Cleaners, Except Maids and Housek...",Cleaners of Vehicles and Equipment,3.6,-5.989386,"Janitors and Cleaners, Except Maids and Housek...","Pesticide Handlers, Sprayers, and Applicators,...",8.3,30.629265
3,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061,"Janitors and Cleaners, Except Maids and Housek...","Grounds Maintenance Workers, All Other",6.5,16.982563
4,"Janitors and Cleaners, Except Maids and Housek...",Shoe and Leather Workers and Repairers,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...",Tree Trimmers and Pruners,10.7,45.716452
5,"Janitors and Cleaners, Except Maids and Housek...","Graders and Sorters, Agricultural Products",0.0,-6.444276,"Janitors and Cleaners, Except Maids and Housek...",Pest Control Workers,8.5,36.087945
6,"Janitors and Cleaners, Except Maids and Housek...",Slaughterers and Meat Packers,3.3,6.520091,"Janitors and Cleaners, Except Maids and Housek...",Construction Laborers,5.4,34.344200
7,"Janitors and Cleaners, Except Maids and Housek...",Manicurists and Pedicurists,19.2,-6.065201,"Janitors and Cleaners, Except Maids and Housek...",Retail Salespersons,-0.6,-7.960576
8,"Janitors and Cleaners, Except Maids and Housek...",Solderers and Brazers,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...",Childcare Workers,1.7,-11.675512
9,"Janitors and Cleaners, Except Maids and Housek...","Cooks, Fast Food",-13.4,-14.329037,"Janitors and Cleaners, Except Maids and Housek...","Laborers and Freight, Stock, and Material Move...",4.2,7.581501


In [387]:
# get_df_comparison(df_onet,df_cps,good_dict,'Accountants',10)